In [1]:
from tqdm import tqdm
from bitstring import BitArray
import numpy as np

In [31]:
def test_hash_int(value):
    
    value = int(value)
    
    phi = (1 + 5 ** 0.5) / 2
    g = int(2 ** 64 /phi)
    value ^= value >> 61 # Does a "bitwise exclusive or". Each bit of the output is the same as the corresponding bit in x if that bit in y is 0, and it's the complement of the bit in x if that bit in y is 1.
    value = (g * value) >> 61
    return value

In [1]:
def fibonacci_hash_float(value):
    
    value = BitArray(float=value, length=64)
    phi = (1 + 5 ** 0.5) / 2
    g = int(2 ** 64 /phi)
    

    value ^= value >> 61
    value = int(g * value.float)
 
    return value

In [45]:
setm = set()
for i in tqdm(np.arange(0,2000, 0.001)):
    setm.add(test_hash_float(i))


100%|██████████| 2000000/2000000 [01:04<00:00, 30931.11it/s]


In [47]:
print(len(setm))
print(len(np.arange(0,2000,0.001)))

2000000
2000000


In [53]:
test_hash_float(0.001144545545555554423)

13048637362605560

In [81]:
def minhash(freqs, times, threshold = None, duration = None):
    old_threshold = threshold
    f = 0
    out = []
    
    for i in range(0,duration//threshold):

        for fr, tm in zip(freqs,times):
            
            if(tm < threshold and tm > (threshold-old_threshold)):
            
                f += (test_hash_float(fr) + test_hash_float(tm))
                
            
        out.append(test_hash_float(f))    
        
        threshold += old_threshold
    
    out = np.min(out)
    
    return out
    
        
    #.... e mo? da valutare....
    

In [93]:
a = np.random.normal(0,30, size = 20)
b = np.random.normal(0, 1049, size = 20)
a = abs(a)
b = abs(b)

In [94]:
minhash(b,a,5,30)

500306653179891231383489537655657044377600

In [75]:
a

array([43.64715179,  5.72406566, 26.80923185, 13.4111356 , 39.76989187,
       21.94160356, 44.85278627,  8.17282054,  1.56883747, 84.97118084,
       17.76048701, 31.7438972 , 29.97554441, 32.52407653, 29.59476827,
        9.96261126,  6.18698109, 11.01834602, 17.2729524 , 31.18411419])

In [2]:
BitArray(float=10.5, length=64)

BitArray('0x4025000000000000')